In [1]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19883 sha256=2d0de702d75439680b0932af7901515a681cffb2958337219f6f49ba63baeb85
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\28\fe\0c\fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack


In [97]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.ensemble import RandomForestClassifier 
from xgboost import XGBClassifier 
from lightgbm import LGBMClassifier as lgb
from vecstack import stacking
import os
import pandas as pd

In [39]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [13]:
os.chdir('/content/gdrive/My Drive/2020-2 주제분석')

In [68]:
train_non = pd.read_csv('train_nonanswer_jo_sum_final.csv',encoding="ISO-8859-1")
test_non = pd.read_csv('test_nonanswer_jo_sum.csv')

In [69]:
submission_non = pd.read_csv('samplesubmission.csv')

In [84]:
train_non_x = train_non.drop(['Party', 'USER_ID'],axis = 1)
train_non_y = train_non['Party']
test_non_x = test_non.drop(['USER_ID'], axis = 1)

In [71]:
# Make train/test split 
# As usual in machine learning task we have X_train, y_train, and X_test 
#X_train, X_val, y_train, y_val = train_test_split(train_non_x, train_non_y, test_size = 0.2, random_state = 0) 

In [85]:
X_train_dummy = pd.get_dummies(train_non_x)
X_test_dummy = pd.get_dummies(test_non_x)

In [86]:
y_train_dummy = pd.get_dummies(train_non_y, drop_first= True)

In [98]:
# Caution! All models and parameter values are just 
# demonstrational and shouldn't be considered as recommended. 
# Initialize 1-st level models. 
models = [RandomForestClassifier(n_estimators = 300, max_depth = 9, criterion = 'entropy', max_features = 'log2'), 
          XGBClassifier(base_score = 0.5, booster = 'gbtree', colsample_bylevel = 1,
                        colsample_bynode = 1, colsample_bytree = 1, gamma = 0, importance_type = 'gain', 
                        max_delta_step = 0, max_depth = 4, learning_rate = 0.1, min_child_weight = 1, 
                        num_parallel_tree = 1, random_state = 0, n_jobs = 0, n_estimators = 100, reg_alpha = 0, reg_lambda = 20,
                        scale_pos_weight = 1, subsample = 1, tree_method = 'exact', validate_parameters = 1, verbosity = 0),
          lgb(boosting_type = 'gbdt', objective = 'binary', learning_rate = 0.01, n_jobs = 5, n_estimators = 400,
              num_leaves = 700, silent = True, max_depth = -1, max_bin = 255, colsample_bytree = 0.64, reg_alpha = 1.4,
              reg_lambda = 1, subsample = 0.7, min_split_gain = 0.5, min_child_weight = 23, min_child_samples = 5)] 

In [99]:
# Compute stacking features 
S_train, S_test = stacking(models, X_train_dummy, y_train_dummy, X_test_dummy, 
                           regression = False, metric = accuracy_score, n_folds = 4, 
                           stratified = True, shuffle = True, random_state = 0) 

In [100]:
# Initialize 2-nd level model 
model = XGBClassifier(base_score = 0.5, booster = 'gbtree', colsample_bylevel = 1,
                        colsample_bynode = 1, colsample_bytree = 1, gamma = 1, importance_type = 'gain',
                        max_delta_step = 0, max_depth = 4, learning_rate = 0.1, min_child_weight = 1, 
                        num_parallel_tree = 1, random_state = 0, n_jobs = 0, n_estimators = 100, reg_alpha = 0, reg_lambda = 20,
                        scale_pos_weight = 1, subsample = 1, tree_method = 'exact', validate_parameters = 1, verbosity = 0)

In [101]:
# Fit 2-nd level model 
model = model.fit(S_train, y_train_dummy) 

In [102]:
# Predict 
y_pred = model.predict(S_test) 

In [103]:
target_data = pd.DataFrame(y_pred)
target_data.columns = ['Predictions']
target_data.loc[target_data["Predictions"] == 0,"Predictions"] = "Democrat"
target_data.loc[target_data["Predictions"] == 1,"Predictions"] = "Republican"
submission = pd.concat([submission_non['USER_ID'],target_data],axis=1)

In [104]:
submission.to_csv("stack_3.csv",header=True,index=False)

In [83]:
# Final prediction score 
print('Final prediction score: [%.8f]' % accuracy_score(y_val, y_pred))

Final prediction score: [0.64807692]


In [ ]:
# Make train/test split 
# As usual in machine learning task we have X_train, y_train, and X_test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0) 

# Caution! All models and parameter values are just 
# demonstrational and shouldn't be considered as recommended. 
# Initialize 1-st level models. 
models = [RandomForestClassifier(n_estimators = 300, max_depth = 9, criterion = 'entropy', max_features = 'log2'), 
          XGBClassifier(base_score = 0.5, booster = 'gbtree', colsample_bylevel = 1,
                        colsample_bynode = 1, colsample_bytree = 1, gamma = 1, gpu_id = -1, importance_type = 'gain',
                        interaction_constraints = ' ', max_delta_step = 0, max_depth = 4, learning_rate = 0.1, min_child_weight = 1, missing = nan, monotone_constraints = '()', 
                        num_parallel_tree = 1, random_state = 0, n_jobs = 0, n_estimators = 100, reg_alpha = 0, reg_lambda = 20,
                        scale_pos_weight = 1, subsample = 1, tree_method = 'exact', validate_parameters = 1, verbosity = None)] 
          
# Compute stacking features 
S_train, S_test = stacking(models, X_train, y_train, X_test, 
                           regression = False, metric = accuracy_score, n_folds = 4, 
                           stratified = True, shuffle = True, random_state = 0, verbose = 2) 

# Initialize 2-nd level model 
model = XGBClassifier(seed = 0, n_jobs = -1, learning_rate = 0.1, n_estimators = 100, max_depth = 3) 

# Fit 2-nd level model 
model = model.fit(S_train, y_train) 
# Predict 
y_pred = model.predict(S_test) 

# Final prediction score 
print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred))

